# Visualizing Neo4j Graph Data Science (GDS) Graphs

In [ ]:
# make sure to install the graphdatascience package
%pip install graphdatascience

## Setup GDS graph

In [ ]:
import os

from graphdatascience import GraphDataScience

# Get Neo4j DB URI, credentials and name from environment if applicable
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = ("neo4j", None)
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

In [ ]:
G = gds.graph.load_cora()

In [ ]:
# Run some algorithms to use later for visualization
gds.pageRank.mutate(G, mutateProperty="pagerank")
gds.louvain.mutate(G, mutateProperty="componentId")

For visualization we recommend limiting the number of nodes to 10k.
A good way to create a smaller subgraph is by using the [sampling](https://neo4j.com/docs/graph-data-science/current/management-ops/graph-creation/sampling/) or [filtering](https://neo4j.com/docs/graph-data-science/current/management-ops/graph-creation/graph-filter/) the GDS graph.

In [ ]:
G_sample, _ = gds.graph.sample.rwr("sampled", G)

In [ ]:
G_sample.node_properties()

## Visualization

In [ ]:
from neo4j_viz.gds import from_gds

# TODO inform about automatic size scaling
VG = from_gds(
    gds,
    G_sample,
    size_property="pagerank",
    additional_node_properties=["componentId", "subject"],
)

In [ ]:
VG.render()

### Adding captions

In [ ]:
for node in VG.nodes:
    node.caption = str(node.subject)

In [ ]:
VG.render()

### Coloring

multiple ways to color

* default colors
* provide custom color map (such as from palettable), such as if you exceed limit

In [ ]:
VG.color_nodes("subject")
VG.render()

#### Custom coloring

In some cases, the default colors are too few.
For example, if you have many communities, you might need a lot more colors.


In [ ]:
# This will warn about reusing colors for multiple components
VG.color_nodes("componentId")

In [ ]:
%pip install matplotlib, palettable

In [ ]:
from palettable.colorbrewer.qualitative import Dark2_7
import matplotlib.colors as mcolors

number_of_components = len({n.componentId for n in VG.nodes})
print(f"Number of components: {number_of_components}")

linear_color_map = Dark2_7.mpl_colormap.resampled(number_of_components)
colors = [mcolors.rgb2hex(linear_color_map(i)) for i in range(number_of_components)]

VG.color_nodes("componentId", colors=colors, override=True)

In [ ]:
VG.render()

### Render options

Besides changing the appearance of our nodes and relationships, we can also modify the rendering itself.
For example, some available options are:

* graph layout
* renderer
* zoom level
* initial position 

In [ ]:
from neo4j_viz import Layout

VG.render(layout=Layout.HIERARCHICAL, initial_zoom=0.1, pan_position=(2000, 0))

## Saving the visualization

In [ ]:
from neo4j_viz.options import Renderer

# Save the visualization to a file
with open("out/cora.html", "w") as f:
    f.write(VG.render(renderer=Renderer.CANVAS).data)

## Cleanup

Lets cleanup the graphs we created in GDS.

In [ ]:
G.drop()
G_sample.drop()